In [ ]:
import torch
from pathlib import Path
import numpy as np
import pandas as pd
import h5py
from tqdm import tqdm

# Load embeddings (TAPE or ESM)

In [ ]:
from tape import ProteinBertModel, TAPETokenizer

MODEL_NAME = "TAPE"
MAGNITUDE_DIM = 768
MODEL_MAX_LENGTH = 8192
embedding_model = ProteinBertModel.from_pretrained("bert-base").eval().cuda()
tokenizer = TAPETokenizer(vocab="iupac")

def get_embedding(sequence, L2_norm=False):
    sequence = sequence.upper()
    token_ids = torch.tensor([tokenizer.encode(sequence)]).cuda()
    with torch.no_grad():
        sequence_output, pooled_output = embedding_model(token_ids)
        sequence_output = sequence_output.mean(dim=1)

    if L2_norm:
        sequence_output /= sequence_output.norm(dim=1)[:, None]
    return sequence_output.cpu().numpy()

In [1]:
import esm

MODEL_NAME = "ESM"
MAGNITUDE_DIM = 1280
MODEL_MAX_LENGTH = 1024
model_location = "models/esm1b_t33_650M_UR50S.pt"
model, alphabet = esm.pretrained.load_model_and_alphabet_local(model_location)

model = model.cuda()
batch_converter = alphabet.get_batch_converter()


def get_embedding(sequence):
    sequence = sequence.upper()

    data = [("0", sequence)]
    batch_labels, batch_strs, batch_tokens = batch_converter(data)

    with torch.no_grad():
        results = model(batch_tokens.cuda(), repr_layers=[33])
    token_embeddings = results["representations"][33]
    # NOTE: token 0 is always a beginning-of-sequence token, so the first residue is token 1.
    sequence_embeddings = (
        token_embeddings[0, 1 : len(sequence) + 1].mean(dim=0).cpu().numpy()
    )

    return sequence_embeddings

/home/ubuntu/anaconda3/lib/python3.8/site-packages/esm/pretrained.py:112: UserWarning: Regression weights not found, predicting contacts will not produce correct results.
  warnings.warn("Regression weights not found, predicting contacts will not produce correct results.")


## Read data in CSV and create h5py

In [1]:
minimum_length = 1
maximum_length = MODEL_MAX_LENGTH - 2 # remove extra tokens for N and C terminal
minimum_property = 0.001
maximum_property = 5000
file_property = Path("protein_data.csv")

df_property = pd.read_csv(file_property)
df_property["SEQ"] = df_property["SEQ"].str.upper()
df_property["SEQ_LEN"] = df_property["SEQ"].apply(len)
df_property["PROPERTY_LABEL"] = df_property["PROPERTY"]

df_property = df_property.query(
    f"({minimum_property} <= PROPERTY_LABEL <= {maximum_property}) \
    & \
    ({minimum_length} <= SEQ_LEN <= {maximum_length})"
)
df_property["PROPERTY_LABEL_log10"] = df_property["PROPERTY"].apply(np.log10)

print(df_property.shape) # check shape/size
df_property.head() # print head of csv

NameError: name 'MODEL_MAX_LENGTH' is not defined

In [ ]:
# save embeddings

sequences = df_property["SEQ"]
labels = np.log(df_property["PROPERTY_LABEL"])

n_documents = labels.size
magnitude_dim = MAGNITUDE_DIM
file_save_h5 = (
    file_property.parent
    / f"{MODEL_NAME}_{minimum_length}_{maximum_length}_{minimum_property}_{maximum_property}_{file_property.stem}_log_lab.hdf5"
)

with h5py.File(str(file_save_h5), "a") as file_handler:
    dset_labels = file_handler.create_dataset(
        "labels", (n_documents,), dtype=labels.dtype
    )
    dset_embeddings = file_handler.create_dataset(
        "embeddings", (n_documents, magnitude_dim), dtype="float32"
    )
    for idx, (sequence, label) in enumerate(
        tqdm(zip(sequences, labels), total=n_documents)
    ):
        dset_labels[idx] = label
        dset_embeddings[idx] = get_embedding(sequence)

 70%|██████▉   | 1894/2711 [13:15<04:13,  3.23it/s]

## Check data distribution

In [ ]:
from matplotlib import pyplot as plt

protyield = np.log(df_abundance["Yield"])

#bins = np.arange(np.min(protyield), np.max(protyield), 2)

plt.hist(protyield, alpha=0.5)
plt.xlabel("Yield (ug/ml)")
plt.ylabel("Frequency")
plt.show()